What are HAIs?
Why are they important?

In [1]:
# Import libraries
import pandas as pd
import os.path

microbiology_events = pd.read_csv("../data/raw/mimic-iii-demo/MICROBIOLOGYEVENTS.csv")
chart_events = pd.read_csv("../data/raw/mimic-iii-demo/CHARTEVENTS.csv")
admission = pd.read_csv("../data/raw/mimic-iii-demo/ADMISSIONS.csv")
patient = pd.read_csv("../data/raw/mimic-iii-demo/PATIENTS.csv")

/Users/isabelzimmerman/.local/share/virtualenvs/isabelzimmerman-n-Uyvjwz/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8,10,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


We'll make a data frame with demographic factors first: age, ethnicity, marital status, etc. 

merge admissions, patients

nrow
nrow distinct
filter <300 years old
select subject_id, insurance, religion, ethnicity, gender, age -> demographics

create_report demographics

In [2]:
from datetime import date 

def calculate_age(dob, dod):
    return dob.year - dod.year - ((dod.month, dod.day) < (dob.month, dob.day))

In [7]:
demographics = pd.merge(admission, patient, on='subject_id')

demographics.sample(3)

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [5]:
demographics["dob"] = pd.to_datetime(demographics["dob"], format='%Y%m%d%')
demographics["dod"] = pd.to_datetime(demographics["dod"], infer_datetime_hformat=True)
demographics["age"] = calculate_age(demographics["dob"], demographics["dod"])



AttributeError: 'Series' object has no attribute 'year'

In [ ]:
demographic_columns = ['subject_id', 'insurance', 'gender', 'religion', 'marital_status', 'ethnicity', 'age']

demographics = demographics.loc[ : , demographic_columns]

A useful library to start an EDA from scratch is Pana

In [17]:
#Corr matrix patient demographic
import pandas_profiling

#pandas_profiling.ProfileReport(microbiology_events)

merge admissions, microbio_events

hadm_id, spec_itemid, spec_type_desc, org_name, ab_name, diagnosis, interpretation

In [20]:
pt_info = pd.merge(admission, microbiology_events, on='hadm_id')
pt_columns = ['subject_id_x', 'hadm_id', 'admission_type', 'admission_location', 'diagnosis', 'spec_itemid', 'org_itemid','interpretation']

pt_info = pt_info.loc[ : , pt_columns]
#pandas_profiling.ProfileReport(patient_info)
pt_info.sample(3)

,subject_id_x,hadm_id,admission_type,admission_location,diagnosis,spec_itemid,org_itemid,interpretation
1998,44228,103379,EMERGENCY,EMERGENCY ROOM ADMIT,CHOLANGITIS,70012,NaN,NaN
1702,42135,117105,EMERGENCY,EMERGENCY ROOM ADMIT,HEPATIC ENCEP,70062,80023.0,S
147,10038,111115,EMERGENCY,EMERGENCY ROOM ADMIT,FAILURE TO THRIVE,70012,80045.0,S


In [16]:
#pandas_profiling.ProfileReport(patient_info)